In [ ]:
import pandas as pd
import re
file = open("DATA.txt", "r",encoding='utf-8')
raw = [t for t in file.readlines()]
raw = raw[3:].copy()
rawtext = "".join(raw)

# 시간 추출
time_p = re.compile('\[(오전 |오후 )+([^]]+)\]') # 시간 추출 패턴
times = re.finditer(time_p, rawtext)
times_loc = [e.span() for e in times]
times = re.finditer(time_p, rawtext)
times_txt = [e.group().replace("[", "")[:-1] for e in times]

#날짜 추출
date_p = re.compile("\d{4}년\s\d{1,2}월\s\d{1,2}일\s\S+요일") # 날짜 추출 패턴
dates = re.finditer(date_p, rawtext)
dates_loc = [e.span() for e in dates]
dates = re.finditer(date_p, rawtext)
dates_txt = [e.group() for e in dates]

# 유저명 추출
name_p = re.compile('\[([^]]+)\] \[오') # 유저명 추출 패턴
names = re.finditer(name_p, rawtext)
names_loc = [e.span() for e in names] # 위치 추출
names = re.finditer(name_p, rawtext)
names_txt = [e.group()[:-2] for e in names] # 텍스트 추출
names_txt = [e.replace("[", "")[:-2] for e in names_txt]

# 메세지 추출
texts = []
for i, tl in enumerate(times_loc):
    tl = tl[1]+1
    try:
        nl = names_loc[i+1][0]-1
    except:
        nl = len(rawtext)
    texts.append((tl, rawtext[tl:nl])) # 시간 패턴의 끝 위치부터 다음 유저명 패턴 시작 전 위치([오 텍스트 빼려고) 까지가 메세지

df = pd.DataFrame(texts) # 데이터프레임 생성
dates_text = []
for num in df[0]: # 저장한 캐릭터 번호를 기준으로 날짜 부여
    for dt, dtl in zip(dates_txt, dates_loc): 
        if num >= dtl[1]: # 날짜 텍스트 위치의 끝번호보다 캐릭터 번호가 크면
            rdt = dt # dt값을 해당 날짜를 기준으로 업데이트
        elif num < dtl[1]:  # 날짜 텍스트 위치의 끝번호보다 캐릭터 번호가 작으면
            break
    dates_text.append(rdt) # 메세지 별 날짜 저장 

df['date']=dates_text
df['time']=times_txt
df['name']=names_txt
df.rename({0:'msgid',1:'msg'}, axis=1, inplace=True)

from datetime import datetime
weekdays = {
    '일요일': 'Sunday',
    '월요일': 'Monday',
    '화요일': 'Tuesday',
    '수요일': 'Wednesday',
    '목요일': 'Thursday',
    '금요일': 'Friday',
    '토요일': 'Saturday'
}

df['datetime'] = df.date + " " + df.time
new_date = []
for dt in df['datetime']:
    for k in weekdays.keys():
        dt = dt.replace(k, weekdays[k])
    dt = dt.replace("오전","AM").replace("오후","PM")
    new_date.append(dt)
df['datetime'] = new_date
df['datetime'] = [datetime.strptime(x, f"%Y년 %m월 %d일 %A %p %I:%M") for x in df.datetime]

# %A: 요일의 전체 이름
# %p: AM 또는 PM
# %I: 12시간제 시간
# %M: 분 (00부터 59)

## text 묶음으로 RAG

In [ ]:
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.prompts import PromptTemplate
# from langchain.memory import ConversationSummaryBufferMemory
# from langchain_community.llms import Ollama
# from langchain.vectorstores import Chroma
# from langchain_huggingface import HuggingFaceEmbeddings
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.document_loaders import DataFrameLoader
# import pandas as pd
# import pickle
# from tqdm import tqdm

# # 1. 문서 준비하기
# texts = df.query('name=="ㄱ✌️우리 예쁜 민쥬✌️"')['msg']
# texts = texts.reset_index(drop=True)
# document = " ".join(texts)

# # # 2. 문서를 작은 조각으로 나누기
# # # 이렇게 하면 나중에 필요한 정보를 더 쉽게 찾을 수 있어요
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
#                                                 chunk_overlap=50,
#                                                 length_function=len,
#                                                 is_seperator_regex=False,) # chunk_size = 문서 조각 1개 내 문자 수
# texts = text_splitter.create_documents(document)

# # 3. 벡터 데이터베이스 만들기
# # 이것은 마치 책의 색인처럼 나중에 정보를 빨리 찾을 수 있게 해줍니다

# embeddings = HuggingFaceEmbeddings(
#     model_name="jhgan/ko-sroberta-multitask",
#     model_kwargs={'device':'cuda'},
#     encode_kwargs={'normalize_embeddings':True,'batch_size': 32})

# with open(file='embeddings_mj.pickle', mode='wb') as f:
#     pickle.dump(embeddings, f)

# max_batch_size=166
# texts = [texts[i:i+max_batch_size] for i in range(0, len(texts), max_batch_size)] # 최대 배치 사이즈 만큼 잘라줌

# db = Chroma.from_texts(texts=texts[0],
#             embedding=embeddings)

# for chunk in tqdm(texts[1:]):
#     db.add_texts(texts=chunk, embedding_function=embeddings)

# # 4. Llama 모델 준비하기
# # 이 모델이 실제로 질문에 답변을 생성할 거예요


# # 5. 질문에 답변하는 함수 만들기


## df -> documnet로 RAG

In [ ]:
df['datetime'] = [str(x) for x in df['datetime']] # documnet 객체에는 str, int, float, bool 만 들어갈 수 있음

In [ ]:
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.prompts import PromptTemplate
# from langchain.memory import ConversationSummaryBufferMemory
# from langchain_community.llms import Ollama
# from langchain.vectorstores import Chroma
# from langchain_huggingface import HuggingFaceEmbeddings
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.document_loaders import DataFrameLoader
# import pandas as pd
# import pickle
# from tqdm import tqdm

# # 1. 문서 준비하기
tardf = df.copy()

# # 2. 문서를 작은 조각으로 나누기
# # 이렇게 하면 나중에 필요한 정보를 더 쉽게 찾을 수 있어요
loader = DataFrameLoader(tardf, page_content_column='msg')
pages = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, # chunk_size: chunk의 최대 크기(length_function로 측정)
                                                chunk_overlap=50, # chunk_size = 문서 조각 1개 내 문자 수
                                                length_function=len,) 
docs = text_splitter.split_documents(pages)

# 3. 벡터 데이터베이스 만들기
# 이것은 마치 책의 색인처럼 나중에 정보를 빨리 찾을 수 있게 해줍니다

embeddings = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-multitask",
    model_kwargs={'device':'cuda'},
    encode_kwargs={'normalize_embeddings':True,'batch_size': 32})

with open(file='embeddings_mj.pickle', mode='wb') as f:
    pickle.dump(embeddings, f)

max_batch_size=166
docs = [docs[i:i+max_batch_size] for i in range(0, len(docs), max_batch_size)] # 최대 배치 사이즈 만큼 잘라줌
db = Chroma.from_documents(documents=docs[0],embedding=embeddings)
for chunk in tqdm(docs[1:]):
    db.add_documents(documents=chunk, embedding_function=embeddings)

In [ ]:
llm = Ollama(model='llama3:8b')
def answer_question(question):
    # 관련 정보 찾기
    docs = db.similarity_search(question)
    context = "\n".join([doc.page_content for doc in docs])
    
    # 프롬프트 만들기
    # 이것은 Llama 모델에게 어떻게 답변해야 할지 알려주는 지시사항이에요
    # prompt = PromptTemplate(
    #     input_variables=["context", "question"],
    #     template="""
    #     텍스트:{context} 
    #     질문: {question}
    #     텍스트는 남자 '민주'가 여자친구 '은영'에게 보낸 메세지입니다.
    #     텍스트를 바탕으로 마치 당신이 '민주' 인 것처럼 질문에 답변해주세요.
    #     답변은 한국어로 해주시고, 답변 시 꼭 이 사람의 말투를 똑같이 흉내내주세요.
    #     \n\n답변:"""
    # )

    prompt = PromptTemplate(
        input_variables=["context", "question"],
        template="""
        문서:{context} 
        질문: {question}
        문서를 참고해 답변해주세요. 답변은 반드시 한국어로 해주세요
        \n\n답변:"""
    )
        # 텍스트는 여자 'USER2'가 남자친구 'USER2' 에게 보낸 메세지입니다.
        # 텍스트를 바탕으로 마치 당신이 'USER2' 인 것처럼 질문에 답변해주세요.
        # 답변은 한국어로 해주시고, USER2의 말투를 똑같이 사용해주세요.
    memory = ConversationSummaryBufferMemory(
    llm= Ollama(model='llama3:8b',temperature=1), 
    memory_key="chat_history",
    return_messages=True,
)
    def load_memory(input):
        print(input)
        return memory.load_memory_variables({})["chat_history"]

    # 답변 생성하기
    chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm
    response = chain.invoke({"question":question,
                             "context":context})

    memory.save_context({"input": question},
                        {"output":response})
    return response

In [ ]:
# 메타데이터 필터링 구현하기...

In [ ]:
question = "USER2가 먹고싶은 음식이 뭐야. 한글로 대답해줘."
answer = answer_question(question)
print(answer)

In [ ]:
# 6. 질문하고 답변 받기
# question = "질문: {USER2의 MBTI를 추측해주세요.} 요구사항: {한글로 작성, 추측한 이유 5개. 원문 텍스트를 근거로 상세 설명.} "
answer = answer_question(question)
print(answer)